https://www.youtube.com/watch?v=6DGYj1EEWOw&t=240s
https://github.com/rohan-paul/LLM-FineTuning-Large-Language-Models/blob/main/Mistral_FineTuning_with_PEFT_and_QLORA.ipynb

In [1]:
pip install -q -U bitsandbytes peft bitsandbytes accelerate trl datasets


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dataclasses import dataclass, field
from datasets import load_dataset
from typing import Optional
import torch
from peft import LoraConfig
from peft import AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer

In [7]:
dataset = load_dataset("json", data_files="IOdata.json")
# Might have to do train/test split

In [9]:
@dataclass
class ScriptArguments:
    """
    These arguments vary depending on how many GPUs you have, what their capacity and features are, and what size model you want to train.
    """
    local_rank: Optional[int] = -1 # -1 means traning is not distributed
    per_device_train_batch_size: Optional[int] = 4 # 4 times num gpus
    per_device_eval_batch_size: Optional[int] = 4 # same thing as above
    gradient_accumulation_steps: Optional[int] = 4 # how many steps until we do a backward pass
    learning_rate: Optional[float] = 2e-5
    max_grad_norm: Optional[float] = 0.3 # max allowed norm for the gradient before backprop
    weight_decay: Optional[int] = 0.01
    lora_alpha: Optional[int] = 16
    lora_dropout: Optional[float] = 0.1
    lora_r: Optional[int] = 32
    max_seq_length: Optional[int] = 512
    model_name: Optional[str] = "mistralai/Mistral-7B-v0.1"
    dataset_name: Optional[str] = None
    use_4bit: Optional[bool] = True
    use_nested_quant: Optional[bool] = False
    bnb_4bit_compute_dtype: Optional[str] = "float16"
    bnb_4bit_quant_type: Optional[str] = "nf4"
    num_train_epochs: Optional[int] = 100
    fp16: Optional[bool] = False
    bf16: Optional[bool] = True
    packing: Optional[bool] = False
    gradient_checkpointing: Optional[bool] = True
    optim: Optional[str] = "paged_adamw_32bit"
    lr_scheduler_type: str = "constant"
    max_steps: int = 1000000
    warmup_ratio: float = 0.03
    group_by_length: bool = True
    save_steps: int = 50
    logging_steps: int = 50
    merge_and_push: Optional[bool] = False
    output_dir: str = "./results_packing"

In [11]:
"""
These two lines shoule be used when executed from a python file, not really an ipynb
takes dataclasse's arguments and converts them to a dictionary
"""
# parser = HfArgumentParser(ScriptArguments)
# script_args = parser.parse_args_into_dataclasses()[0]

In [12]:
# Do this instead in a jupyter notebook
script_args = ScriptArguments(
    local_rank=-1,
    per_device_train_batch_size=1,  # custom value
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,  # custom value
    max_grad_norm=0.3,
    weight_decay=0.01,
    lora_alpha=16,
    lora_dropout=0.1,
    lora_r=32,
    max_seq_length=512,
    model_name="mistralai/Mistral-7B-v0.1",
    dataset_name="",
    use_4bit=True,
    use_nested_quant=False,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4",
    num_train_epochs=100,
    fp16=True,
    bf16=False,
    packing=False,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    lr_scheduler_type="constant",
    max_steps=1000000,
    warmup_ratio=0.03,
    group_by_length=True,
    save_steps=50,
    logging_steps=50,
    merge_and_push=False,
    output_dir="./results_packing"
)

In [ ]:
def create_and_prepare_model(args):
    compute_dtype = getattr(torch, args.bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=args.use_4bit,
        bnb_4bit_quant_type=args.bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=args.use_nested_quant,
    )

    if compute_dtype == torch.float16 and args.use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
            print("=" * 80)

    # Load the entire model on the GPU 0
    # switch to `device_map = "auto"` for multi-GPU
    device_map = {"": 0}

    model = AutoModelForCausalLM.from_pretrained(
        args.model_name,
        quantization_config=bnb_config,
        device_map=device_map,
        # use_auth_token=True,
        # revision="refs/pr/35"
    )

    #### LLAMA STUFF
    # check: https://github.com/huggingface/transformers/pull/24906
    model.config.pretraining_tp = 1
    # model.config.
    #### LLAMA STUFF
    model.config.window = 256

    peft_config = LoraConfig(
        lora_alpha=script_args.lora_alpha,
        lora_dropout=script_args.lora_dropout,
        # target_modules=["query_key_value"],
        r=script_args.lora_r,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],
    )

    tokenizer = AutoTokenizer.from_pretrained(script_args.model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token

    return model, peft_config, tokenizer